In [1]:
import warnings
warnings.filterwarnings('ignore')
import gzip
import pandas as pd
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import plotly_express as px
import tqdm
from tqdm._tqdm_notebook import tqdm_notebook
import folium
import shapely
import os
geopy.geocoders.options.default_user_agent = "test_user"
from faker import Faker

In [2]:
conda install -c conda-forge faker 

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [3]:
#reimport from xls 
import pandas as pd
LondonCheckinsClss = pd.read_excel (r'LondonCheckinsBusiness.xlsx')
LondonCheckinsClss.head(2)

,user,check_in_time,latitude,longitude,location_id,checkin_Date,checkin_time,checkin_year,checkin_month,checkin_day,checkin_hour,checkin_weekday,Address,Location,city_district
0,72,2010-04-18T07:04:21Z,51.442041,-0.004450,808780,2010-04-18,07:04:21,2010,4,18,7,Sunday,"Sandhurst Road, Catford South, London Borough ...",Sandhurst Road,London Borough of Lewisham
1,72,2010-04-16T16:33:11Z,51.375217,-0.103929,587260,2010-04-16,16:33:11,2010,4,16,16,Friday,"Centrale, Tamworth Road, Broad Green, London B...",Centrale,London Borough of Croydon


In [4]:
# removing columns for Classification
LondonCheckinsClss = LondonCheckinsClss.drop(['location_id','check_in_time','checkin_Date'
                                              ,'checkin_time','checkin_year','checkin_day','latitude','longitude'
                                              ,'city_district','Location'
                                             # ,'checkin_month','checkin_weekday','checkin_hour','Address'
                                             # ,'latitude','longitude','user'
                                             ], axis=1)
LondonCheckinsClss.head(3)

,user,checkin_month,checkin_hour,checkin_weekday,Address
0,72,4,7,Sunday,"Sandhurst Road, Catford South, London Borough ..."
1,72,4,16,Friday,"Centrale, Tamworth Road, Broad Green, London B..."
2,72,4,16,Friday,"Centrale, Tamworth Road, Broad Green, London B..."


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn import model_selection
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, zero_one_loss

In [6]:
#one hot encoding for month and weekday 
LondonCheckinsClss = pd.get_dummies(LondonCheckinsClss, columns=['checkin_month'], drop_first=True, prefix='month')
LondonCheckinsClss = pd.get_dummies(LondonCheckinsClss, columns=['checkin_weekday'], drop_first=True, prefix='wkday')
LondonCheckinsClss = pd.get_dummies(LondonCheckinsClss, columns=['checkin_hour'], drop_first=True, prefix='hour')
LondonCheckinsClss = pd.get_dummies(LondonCheckinsClss, columns=['Address'], drop_first=True, prefix='Address')
LondonCheckinsClss.head()

,user,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,...,"Address_West Norwood, Hannen Road, West Dulwich, West Norwood, London Borough of Lambeth, London, Greater London, England, SE27 0DT, United Kingdom","Address_West Wickham, Rays Road, West Wickham, London Borough of Bromley, London, Greater London, England, BR4 0EH, United Kingdom","Address_Whitgift Avenue, Haling Park Road, Waddon, London Borough of Croydon, London, Greater London, England, CR2 6NF, United Kingdom","Address_Whitgift Centre, Sydenham Road, Broad Green, London Borough of Croydon, London, Greater London, England, CR0 2YG, United Kingdom","Address_Widmore, London Borough of Bromley, London, Greater London, England, BR2 9PF, United Kingdom","Address_Wing Yip, 544, Purley Way, Waddon, London Borough of Croydon, London, Greater London, England, CR0 4RE, United Kingdom","Address_Woodland Road, London Borough of Lambeth, London, Greater London, England, SE19, United Kingdom","Address_York Avenue, Pound Place, Longlands, London Borough of Bexley, London, Greater London, England, DA15 7LJ, United Kingdom","Address_Zizzi, 11 - 13, Widmore Road, St Marks Square, Plaistow, London Borough of Bromley, London, Greater London, England, BR1 1RL, United Kingdom","Address_Zizzi, 57-59, South End, Waddon, London Borough of Croydon, London, Greater London, England, CR0 1BF, United Kingdom"
0,72,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,72,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,72,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,72,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,72,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##  Applying Logistic regression on non-private data

In [7]:
import numpy as np
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    LondonCheckinsClss.drop('user', axis=1), LondonCheckinsClss['user'],
      test_size=0.33, random_state=2021)

In [8]:
NonPrivateClss = LogisticRegression(solver="lbfgs")
NonPrivateClss.fit(X_train, y_train)
NonPriv_pred_clss = NonPrivateClss.predict(X_test)
NonPriv_pred_train_clss = NonPrivateClss.predict(X_train)
clss_error = zero_one_loss(y_test, NonPriv_pred_clss)

print("Training set score: %.2f%%" % (NonPrivateClss.score(X_train, y_train) * 100))
print("Test set score: %.2f%%" % (NonPrivateClss.score(X_test, y_test) * 100))
print("Error: %.2f%%" % ((clss_error) * 100))

Training set score: 75.62%
Test set score: 52.60%
Error: 47.40%


# Applying Dummy Locations (Spoofing) - rudimentary approach

In [9]:
LondonCheckinsClss.user.unique()

array([    72,    552,    726,    821,    939,   1170,   1384,   1494,
         1815,   1886,   2025,   2786,   4535,   4633,   4701,   4739,
         4754,   4894,   4899,   4900,   6304,   7162,   7199,   7239,
         7240,   7431,   7563,   7675,   7922,   8382,   8386,   8419,
         8457,   9647,  10657,  10670,  10674,  10683,  10698,  10699,
        10708,  10915,  11543,  12543,  14476,  14509,  15095,  15673,
        15794,  17006,  17052,  17986,  19470,  19659,  19709,  19973,
        20093,  20409,  20471,  21462,  22150,  23730,  23858,  23906,
        23937,  26668,  27044,  28170,  29522,  29641,  30565,  30598,
        31907,  32842,  33880,  34115,  34671,  35154,  35963,  35967,
        35968,  36596,  38293,  39140,  39201,  39940,  40091,  40301,
        40304,  40377,  40538,  41212,  41435,  42670,  46747,  48334,
        48553,  48631,  49451,  49454,  50139,  50164,  50238,  50247,
        51523,  51933,  52944,  53731,  55200,  55628,  56027,  57033,
      

In [10]:
#https://faker.readthedocs.io/en/master/
#https://faker.readthedocs.io/en/master/locales/en_GB.html#faker-providers-address
from faker import Faker 
fake = Faker('en_GB')
import json        
from random import randint  
import random
  
def input_data(x): 
  
    # dictionary 
    spoof_data ={} 
    for i in range(0, x): 
        spoof_data[i]={} 
        spoof_data[i]['user']= randint(72, 196140)
        spoof_data[i]['latitude']= random.uniform(51.345375, 51.444771) 
        spoof_data[i]['longitude']= random.uniform(-0.123386, 0.112477) 
        spoof_data[i]['checkin_month']= randint(1, 12)
        spoof_data[i]['checkin_hour']= randint(0, 24)
#        spoof_data[i]['latitude']= str(fake.latitude()) 
#        spoof_data[i]['longitude']= str(fake.longitude()) 
#        spoof_data[i]['address']= fake.address() 
#        spoof_data[i]['Country']= str(fake.current_country())
    #print(spoof_data) 

    with open('spoof_data.json', 'w') as fp: 
        json.dump(spoof_data, fp) 
      
  
def main(): 
    number_of_users = 100
    input_data(number_of_users) 
main()

In [11]:
df=pd.read_json('spoof_data.json')
df = df.transpose()
df

,user,latitude,longitude,checkin_month,checkin_hour
0,71176.0,51.391721,0.034478,6.0,13.0
1,55051.0,51.378729,-0.090394,6.0,18.0
2,103207.0,51.375429,0.110839,1.0,19.0
3,117669.0,51.427863,-0.041034,8.0,2.0
4,122331.0,51.383930,0.098718,5.0,7.0
...,...,...,...,...,...
95,57658.0,51.438754,-0.016769,6.0,18.0
96,35924.0,51.433555,0.087100,5.0,17.0
97,151360.0,51.418463,0.106368,12.0,22.0
98,97390.0,51.430603,-0.067294,7.0,7.0


In [12]:
df['user'] = df['user'].astype('int')
df['latitude'] = df['latitude'].astype('float64')
df['longitude'] = df['longitude'].astype('float64')
df['checkin_month'] = df['checkin_month'].astype('int')
df['checkin_hour'] = df['checkin_hour'].astype('int')
df.dtypes

user               int32
latitude         float64
longitude        float64
checkin_month      int32
checkin_hour       int32
dtype: object

In [13]:
FilterLatsSpoof = df[(df['latitude']>= 51.345375) & (df['latitude']<= 51.444771)]
FakeCheckins = FilterLatsSpoof[(FilterLatsSpoof['longitude']>= -0.123386) & (FilterLatsSpoof['longitude']<=0.112477)]
FakeCheckins.head(1000)

,user,latitude,longitude,checkin_month,checkin_hour
0,71176,51.391721,0.034478,6,13
1,55051,51.378729,-0.090394,6,18
2,103207,51.375429,0.110839,1,19
3,117669,51.427863,-0.041034,8,2
4,122331,51.383930,0.098718,5,7
...,...,...,...,...,...
95,57658,51.438754,-0.016769,6,18
96,35924,51.433555,0.087100,5,17
97,151360,51.418463,0.106368,12,22
98,97390,51.430603,-0.067294,7,7


In [14]:
#test for one lat & long
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent= "Reverse Geocoding", timeout=10)

In [15]:
#add full address for analysis
FakeCheckins["Address"] = FakeCheckins.apply(lambda x:geolocator.reverse(str(x["latitude"])+", "+str(x["longitude"])).address, axis=1)

In [16]:
FakeCheckins.head(2)

,user,latitude,longitude,checkin_month,checkin_hour,Address
0,71176,51.391721,0.034478,6,13,"Walpole Road, Southborough, London Borough of ..."
1,55051,51.378729,-0.090394,6,18,"Yardmaster House, 11, Cross Road, Addiscombe, ..."


## Map the two datasets

In [17]:
#reimport from xls 
import pandas as pd
LondonCheckinsMap = pd.read_excel (r'LondonCheckinsBusiness.xlsx')
LondonCheckinsMap.head(2)

,user,check_in_time,latitude,longitude,location_id,checkin_Date,checkin_time,checkin_year,checkin_month,checkin_day,checkin_hour,checkin_weekday,Address,Location,city_district
0,72,2010-04-18T07:04:21Z,51.442041,-0.004450,808780,2010-04-18,07:04:21,2010,4,18,7,Sunday,"Sandhurst Road, Catford South, London Borough ...",Sandhurst Road,London Borough of Lewisham
1,72,2010-04-16T16:33:11Z,51.375217,-0.103929,587260,2010-04-16,16:33:11,2010,4,16,16,Friday,"Centrale, Tamworth Road, Broad Green, London B...",Centrale,London Borough of Croydon


In [27]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent= "Reverse Geocoding", timeout=10)
df["address"] = df.apply(lambda x:geolocator.reverse(str(x["latitude"])+", "+str(x["longitude"])).address, axis=1)

#add the spoof data
fig = px.scatter_mapbox(df, lat="latitude", lon="longitude", hover_name="address", 
                       color_discrete_sequence=["red"], zoom=10, width=1000, height=600)
#add actual user datapoints
fig.add_trace(px.scatter_mapbox(LondonCheckinsMap, lat="latitude", lon="longitude", hover_name="user", 
                        color_discrete_sequence=["black"],size_max=30, zoom=10, width=1000, height=600).data[0])

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

## Add spoof data to original dataset

In [19]:
FakeCheckins.head(2)

,user,latitude,longitude,checkin_month,checkin_hour,Address
0,71176,51.391721,0.034478,6,13,"Walpole Road, Southborough, London Borough of ..."
1,55051,51.378729,-0.090394,6,18,"Yardmaster House, 11, Cross Road, Addiscombe, ..."


In [20]:
LondonCheckinsMap = LondonCheckinsMap.drop(['location_id','check_in_time','checkin_Date'
                                              ,'checkin_time','checkin_year','checkin_day'
                                              ,'city_district','checkin_weekday','Location'
                                             # ,'checkin_month','checkin_hour'
                                             # ,'latitude','longitude','user','Address'
                                             ], axis=1)
LondonCheckinsMap.head(3)

,user,latitude,longitude,checkin_month,checkin_hour,Address
0,72,51.442041,-0.004450,4,7,"Sandhurst Road, Catford South, London Borough ..."
1,72,51.375217,-0.103929,4,16,"Centrale, Tamworth Road, Broad Green, London B..."
2,72,51.375217,-0.103929,4,16,"Centrale, Tamworth Road, Broad Green, London B..."


In [21]:
PrivateSpoofedDATA = pd.concat([LondonCheckinsMap, FakeCheckins])

In [22]:
PrivateSpoofedDATA.head(1)

,user,latitude,longitude,checkin_month,checkin_hour,Address
0,72,51.442041,-0.00445,4,7,"Sandhurst Road, Catford South, London Borough ..."


In [23]:
#one hot encoding for month and weekday 
df_spoof = pd.get_dummies(PrivateSpoofedDATA, columns=['checkin_month'], drop_first=True, prefix='month')
df_spoof = pd.get_dummies(PrivateSpoofedDATA, columns=['checkin_hour'], drop_first=True, prefix='hour')
df_spoof = pd.get_dummies(PrivateSpoofedDATA, columns=['Address'], drop_first=True, prefix='Address')
df_spoof.head()

,user,latitude,longitude,checkin_month,checkin_hour,"Address_131, Lower Road, St Mary Cray, London Borough of Bromley, London, Greater London, England, BR5 4AJ, United Kingdom","Address_14, The Oval, The Hollies, Blackfen, London Borough of Bexley, London, Greater London, England, DA15 9ES, United Kingdom","Address_2, Kidderminster Road, Broad Green, London Borough of Croydon, London, Greater London, England, CR0 2UE, United Kingdom","Address_2, Waldram Park Road, Forest Hill, London Borough of Lewisham, London, Greater London, England, SE23 2ND, United Kingdom","Address_203, Brighton Road, London Borough of Croydon, London, Greater London, England, CR2 6EJ, United Kingdom",...,"Address_Wing Yip, 544, Purley Way, Waddon, London Borough of Croydon, London, Greater London, England, CR0 4RE, United Kingdom","Address_Woodland Road, London Borough of Lambeth, London, Greater London, England, SE19, United Kingdom","Address_Woodland Way, Shirley, London Borough of Croydon, London, Greater London, England, CR0 7UB, United Kingdom","Address_Woodpecker, Beulah Hill, Upper Norwood, London Borough of Croydon, London, Greater London, England, SE19 3LR, United Kingdom","Address_Yardmaster House, 11, Cross Road, Addiscombe, London Borough of Croydon, London, Greater London, England, CR0 6TA, United Kingdom","Address_Yester Drive, Elmstead, London Borough of Bromley, London, Greater London, England, BR7 5LR, United Kingdom","Address_Yewdale Close, Shortlands, London Borough of Bromley, London, Greater London, England, BR1 4JJ, United Kingdom","Address_York Avenue, Pound Place, Longlands, London Borough of Bexley, London, Greater London, England, DA15 7LJ, United Kingdom","Address_Zizzi, 11 - 13, Widmore Road, St Marks Square, Plaistow, London Borough of Bromley, London, Greater London, England, BR1 1RL, United Kingdom","Address_Zizzi, 57-59, South End, Waddon, London Borough of Croydon, London, Greater London, England, CR0 1BF, United Kingdom"
0,72,51.442041,-0.004450,4,7,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,72,51.375217,-0.103929,4,16,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,72,51.375217,-0.103929,4,16,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,72,51.374846,-0.103378,4,16,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,72,51.379984,-0.101773,4,16,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
import numpy as np
from sklearn.model_selection import train_test_split
X_trainS, X_testS, y_trainS, y_testS = train_test_split(
    df_spoof.drop('user', axis=1), df_spoof['user'],
      test_size=0.33, random_state=2021)

In [25]:
SPOOFClss = LogisticRegression()
SPOOFClss.fit(X_trainS, y_trainS)
SPOOF_pred_clss = SPOOFClss.predict(X_testS)
SPOOF_pred_train_clss = SPOOFClss.predict(X_trainS)
clss_error = zero_one_loss(y_testS, SPOOF_pred_clss)

print("Training set score: %.2f%%" % (SPOOFClss.score(X_trainS, y_trainS) * 100))
print("Test set score: %.2f%%" % (SPOOFClss.score(X_testS, y_testS) * 100))
print("Error: %.2f%%" % ((clss_error) * 100))

Training set score: 27.22%
Test set score: 22.33%
Error: 77.67%
